**INTRO PLUS TITLE**

In [1]:
library(tidyverse)
library(tidymodels)
library(repr)
library(broom)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

**PREAMBLE FOR WHAT OUR CODING IS ABOUT**

In [2]:
turkish_data <- read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00638/Acoustic%20Features.csv")
colnames(turkish_data) <- c("Class", "RMS_Energy_Mean", "Low_Energy_Mean", "Fluctuation_Mean", "Tempo_Mean", 
                            "MFCC_Mean1", "MFCC_Mean2", "MFCC_Mean3", "MFCC_Mean4", "MFCC_Mean5", "MFCC_Mean6",
                            "MFCC_Mean7", "MFCC_Mean8", "MFCC_Mean9", "MFCC_Mean10", "MFCC_Mean11", "MFCC_Mean12",
                            "MFCC_Mean13", "Roughness_Mean", "Roughness_Slope", "Zero_Crossing_Rate_Mean",
                            "Attack_Time_Mean", "Attack_Time_Slope", "Roll_Off_Mean", "Event_Density_Mean", "Pulse_Clarity_Mean",
                            "Brightness_Mean", "Spectral_Centroid_Mean", "Spectral_Spread_Mean", "Spectral_Skewness_Mean",
                            "Spectral_Kurtosis_Mean", "Spectral_Flatness_Mean", "Entropy_of_Spectrum_Mean", "Chromagram_Mean1",
                            "Chromagram_Mean2", "Chromagram_Mean3", "Chromagram_Mean4", "Chromagram_Mean5", "Chromagram_Mean6",
                            "Chromagram_Mean7", "Chromagram_Mean8", "Chromagram_Mean9", "Chromagram_Mean10", "Chromagram_Mean11",
                            "Chromagram_Mean12", "Harmonic_Change_Detection_Function_Mean", "Harmonic_Change_Detection_Function_Std",
                            "Harmonic_Change_Detection_Function_Slope", "Harmonic_Change_Detection_Function_Period_Freq", 
                            "Harmonic_Change_Detection_Function_Period_Amp", "Harmonic_Change_Detection_Function_Period_Entropy")
turkish_data <- mutate(turkish_data, Class = as_factor(Class))
head(turkish_data)
check_amounts <- turkish_data |>
    group_by(Class) |>
    summarise(count = n())
check_amounts

Rows: 400 Columns: 51
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (1): Class
dbl (50): _RMSenergy_Mean, _Lowenergy_Mean, _Fluctuation_Mean, _Tempo_Mean, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Class,RMS_Energy_Mean,Low_Energy_Mean,Fluctuation_Mean,Tempo_Mean,MFCC_Mean1,MFCC_Mean2,MFCC_Mean3,MFCC_Mean4,MFCC_Mean5,⋯,Chromagram_Mean9,Chromagram_Mean10,Chromagram_Mean11,Chromagram_Mean12,Harmonic_Change_Detection_Function_Mean,Harmonic_Change_Detection_Function_Std,Harmonic_Change_Detection_Function_Slope,Harmonic_Change_Detection_Function_Period_Freq,Harmonic_Change_Detection_Function_Period_Amp,Harmonic_Change_Detection_Function_Period_Entropy
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
relax,0.052,0.591,9.136,130.043,3.997,0.363,0.887,0.078,0.221,⋯,0.426,1.000,0.008,0.101,0.316,0.261,0.018,1.035,0.593,0.970
relax,0.125,0.439,6.680,142.240,4.058,0.516,0.785,0.397,0.556,⋯,0.002,1.000,0.000,0.984,0.285,0.211,-0.082,3.364,0.702,0.967
relax,0.046,0.639,10.578,188.154,2.775,0.903,0.502,0.329,0.287,⋯,0.184,0.746,0.016,1.000,0.413,0.299,0.134,1.682,0.692,0.963
relax,0.135,0.603,10.442,65.991,2.841,1.552,0.612,0.351,0.011,⋯,0.038,1.000,0.161,0.757,0.422,0.265,0.042,0.354,0.743,0.968
relax,0.066,0.591,9.769,88.890,3.217,0.228,0.814,0.096,0.434,⋯,0.004,0.404,1.000,0.001,0.345,0.261,0.089,0.748,0.674,0.957
relax,0.137,0.510,7.664,116.044,2.313,-0.082,0.365,0.033,-0.136,⋯,0.064,0.391,0.102,0.000,0.280,0.240,0.355,1.682,0.662,0.958


Class,count
<fct>,<int>
relax,100
happy,100
sad,100
angry,100


In [3]:
set.seed(6969)

turkish_split <- initial_split(turkish_data, prop = 0.8, strata = Class)
turkish_training <- training(turkish_split)
turkish_testing <- training(turkish_split)

**CROSS VALIDATION**

In [4]:
set.seed(6969)

turkish_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
    set_engine("kknn") |>
    set_mode("classification")

turkish_recipe <- recipe(Class ~ Low_Energy_Mean + Zero_Crossing_Rate_Mean + Attack_Time_Mean + 
                         Pulse_Clarity_Mean + Spectral_Spread_Mean + Harmonic_Change_Detection_Function_Mean,
                         data = turkish_training) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())

turkish_vfold <- vfold_cv(turkish_training, v = 8, strata = Class)

gridvals <- tibble(neighbors = seq(from = 1, to = 80, by = 1))

k_min <- workflow() |>
    add_recipe(turkish_recipe) |>
    add_model(turkish_tune) |>
    tune_grid(resamples = turkish_vfold, grid = gridvals) |>
    collect_metrics() |>
    filter(.metric == "accuracy") |>
    arrange(desc(mean)) |>
    slice(1) |>
    select(neighbors) |>
    pull()

**PRE-MODEL BUILDING**

In [5]:
set.seed(6969)

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = k_min) |>
    set_engine("kknn") |>
    set_mode("classification")

turkish_fit <- workflow() |>
    add_recipe(turkish_recipe) |>
    add_model(knn_spec) |>
    fit(data = turkish_training)

turkish_predictions <- predict(turkish_fit, turkish_testing) |>
    bind_cols(turkish_testing)

turkish_accuracy <- turkish_predictions |>
    metrics(truth = Class, estimate = .pred_class) |>
    filter(.metric == "accuracy")

turkish_mat <- turkish_predictions |>
    conf_mat(truth = Class, estimate = .pred_class)

k_min
turkish_accuracy
turkish_mat

[1] 4

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,multiclass,0.875


          Truth
Prediction relax happy sad angry
     relax    70     2  11     2
     happy     1    76   2     1
     sad       6     2  62     5
     angry     3     0   5    72

**TALK ABOUT THE MODEL**

**GRAPH EXPLINATION AND CONCLUSION**